# 1. Neural Networks
## 1.1 Visualizing the data

In [ ]:
import os
import random
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import scipy.optimize as opt
from pathlib import Path

data_file_1 = Path(os.path.abspath("")) / "data" / "ex4data1.mat"
data_set_1 = sio.loadmat(data_file_1)

# every row of X is a flattened 20 * 20 image matrix
# every row of y is a digit index, where [1, ..., 9] means digit [1, ..., 9], [10] means digit [0]
X = np.array(data_set_1["X"])
y = np.array(data_set_1["y"])

print(X)
print(y)

## 1.2 Model representation

In [ ]:
weights_file = Path(os.path.abspath("")) / "data" / "ex4weights.mat"
weights = sio.loadmat(weights_file)
theta_1 = weights["Theta1"]
theta_2 = weights["Theta2"]
print(theta_1)
print(theta_2)

## 1.3 Feedforward and cost function